In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'3l1f1xn1'

In [3]:
wandb.init(id='1rfauwvw', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 26.6 s, sys: 1.02 s, total: 27.6 s
Wall time: 27.7 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [9]:
set_lr(42)

In [10]:
model_stem = "11.16"
START_E = 118

In [11]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 20 else 3e-4 if e < 100 else 1e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 118

{'trn_control_loss': 0.01544809, 'trn_headings_loss': 0.00088442, 'trn_curvatures_loss': 0.00289889, 'consistency losses/trn_steer_cost': 0.00510401, 'consistency losses/trn_te_loss': 24.52098767, 'aux losses/trn_pitch_loss': 1.18341827, 'aux losses/trn_yaw_loss': 0.28734159, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 3628.92977428, 'consistency losses/trn_torque_delta_loss': 40.45618938, 'logistical/obs_consumed_per_second': 195.37007874, 'logistical/obs_generated_per_second': 169.41732283, 'logistical/slowest_runner_obs_per_sec': 10.62992126, 'logistical/data_consumption_ratio': 1.15804463, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.22120285, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 196.2421875, 'logistical/obs_generated_per_second': 190.8203125, 'logistical/slowest_runner_obs_per_sec': 12.0078125, 'logistical/data_consumption_ratio': 1.0295213, 'logistical/manual_trai

{'logistical/obs_consumed_per_second': 194.8671875, 'logistical/obs_generated_per_second': 193.3984375, 'logistical/slowest_runner_obs_per_sec': 13.2734375, 'logistical/data_consumption_ratio': 1.00820761, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01643446, 'trn_headings_loss': 0.00074644, 'trn_curvatures_loss': 0.00270492, 'consistency losses/trn_steer_cost': 0.00525645, 'consistency losses/trn_te_loss': 23.95374928, 'aux losses/trn_pitch_loss': 1.21418762, 'aux losses/trn_yaw_loss': 0.22874212, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 3113.6467905, 'consistency losses/trn_torque_delta_loss': 31.35266818, 'logistical/max_param': 16.22092247, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 191.7734375, 'logistical/obs_generated_per_second': 187.5625, 'logistical/slowest_runner_obs_per_sec': 12.3515625, 'logistical/data_consumption_ratio': 1.02352211, 'logistical/manual_train_pause': 0.0, 'trn_co

{'logistical/obs_consumed_per_second': 195.3046875, 'logistical/obs_generated_per_second': 198.21875, 'logistical/slowest_runner_obs_per_sec': 13.140625, 'logistical/data_consumption_ratio': 0.98622717, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0183568, 'trn_headings_loss': 0.0009271, 'trn_curvatures_loss': 0.00294207, 'consistency losses/trn_steer_cost': 0.0049043, 'consistency losses/trn_te_loss': 23.94438757, 'aux losses/trn_pitch_loss': 1.39402771, 'aux losses/trn_yaw_loss': 0.29698086, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3133.19055271, 'consistency losses/trn_torque_delta_loss': 28.99506513, 'logistical/max_param': 16.22173882, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 198.0390625, 'logistical/obs_generated_per_second': 187.3984375, 'logistical/slowest_runner_obs_per_sec': 13.25, 'logistical/data_consumption_ratio': 1.05850441, 'logistical/manual_train_pause': 0.0, 'trn_con

{'logistical/obs_consumed_per_second': 192.8515625, 'logistical/obs_generated_per_second': 182.4609375, 'logistical/slowest_runner_obs_per_sec': 11.0078125, 'logistical/data_consumption_ratio': 1.06011974, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01805305, 'trn_headings_loss': 0.00088887, 'trn_curvatures_loss': 0.00322954, 'consistency losses/trn_steer_cost': 0.00671035, 'consistency losses/trn_te_loss': 23.07809342, 'aux losses/trn_pitch_loss': 1.27370453, 'aux losses/trn_yaw_loss': 0.35665989, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 3557.69023037, 'consistency losses/trn_torque_delta_loss': 38.65220095, 'logistical/max_param': 16.23198509, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 196.4609375, 'logistical/obs_generated_per_second': 169.0390625, 'logistical/slowest_runner_obs_per_sec': 8.6796875, 'logistical/data_consumption_ratio': 1.16697586, 'logistical/manual_train_pause': 0.0, 'trn

{'logistical/obs_consumed_per_second': 196.265625, 'logistical/obs_generated_per_second': 195.6484375, 'logistical/slowest_runner_obs_per_sec': 7.0, 'logistical/data_consumption_ratio': 1.00479308, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01497078, 'trn_headings_loss': 0.00076371, 'trn_curvatures_loss': 0.00290059, 'consistency losses/trn_steer_cost': 0.00544815, 'consistency losses/trn_te_loss': 23.346909, 'aux losses/trn_pitch_loss': 1.04681015, 'aux losses/trn_yaw_loss': 0.26023912, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3189.24544525, 'consistency losses/trn_torque_delta_loss': 36.46941483, 'logistical/max_param': 16.23283768, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 197.375, 'logistical/obs_generated_per_second': 178.796875, 'logistical/slowest_runner_obs_per_sec': 7.0, 'logistical/data_consumption_ratio': 1.10519366, 'logistical/manual_train_pause': 0.0, 'trn_control_loss':

...

{'logistical/obs_consumed_per_second': 193.015625, 'logistical/obs_generated_per_second': 188.09375, 'logistical/slowest_runner_obs_per_sec': 11.125, 'logistical/data_consumption_ratio': 1.02588998, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0210152, 'trn_headings_loss': 0.00102907, 'trn_curvatures_loss': 0.00403092, 'consistency losses/trn_steer_cost': 0.00619079, 'consistency losses/trn_te_loss': 24.31170762, 'aux losses/trn_pitch_loss': 1.46529388, 'aux losses/trn_yaw_loss': 0.28181887, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 3614.64112282, 'consistency losses/trn_torque_delta_loss': 38.78984749, 'logistical/max_param': 16.2279644, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 199.6640625, 'logistical/obs_generated_per_second': 183.09375, 'logistical/slowest_runner_obs_per_sec': 10.1015625, 'logistical/data_consumption_ratio': 1.09296458, 'logistical/manual_train_pause': 0.0, 'trn_control_l

{'logistical/obs_consumed_per_second': 175.0, 'logistical/obs_generated_per_second': 203.0, 'logistical/slowest_runner_obs_per_sec': 14.0, 'logistical/data_consumption_ratio': 0.86385726, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 129

{'trn_control_loss': 0.01585579, 'trn_headings_loss': 0.00074734, 'trn_curvatures_loss': 0.00272822, 'consistency losses/trn_steer_cost': 0.00560069, 'consistency losses/trn_te_loss': 24.01913178, 'aux losses/trn_pitch_loss': 1.2138443, 'aux losses/trn_yaw_loss': 0.3254838, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 3479.9509325, 'consistency losses/trn_torque_delta_loss': 35.50580554, 'logistical/obs_consumed_per_second': 193.01574803, 'logistical/obs_generated_per_second': 179.51968504, 'logistical/slowest_runner_obs_per_sec': 10.60629921, 'logistical/data_consumption_ratio': 1.07825709, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.21936607, 'logistical/lr': 0.

{'logistical/obs_consumed_per_second': 198.046875, 'logistical/obs_generated_per_second': 190.1328125, 'logistical/slowest_runner_obs_per_sec': 13.375, 'logistical/data_consumption_ratio': 1.0439922, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02558139, 'trn_headings_loss': 0.00113979, 'trn_curvatures_loss': 0.00381693, 'consistency losses/trn_steer_cost': 0.00741066, 'consistency losses/trn_te_loss': 23.28584544, 'aux losses/trn_pitch_loss': 1.2756424, 'aux losses/trn_yaw_loss': 0.29824257, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.96875, 'consistency losses/trn_torque_loss': 3170.08216, 'consistency losses/trn_torque_delta_loss': 33.8124121, 'logistical/max_param': 16.22337151, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 194.546875, 'logistical/obs_generated_per_second': 194.34375, 'logistical/slowest_runner_obs_per_sec': 13.1171875, 'logistical/data_consumption_ratio': 1.00268021, 'logistical/manual_train_pause': 0.0, 'trn_control_

{'logistical/obs_consumed_per_second': 197.5, 'logistical/obs_generated_per_second': 177.7265625, 'logistical/slowest_runner_obs_per_sec': 8.2734375, 'logistical/data_consumption_ratio': 1.11639192, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01617464, 'trn_headings_loss': 0.0007864, 'trn_curvatures_loss': 0.00287905, 'consistency losses/trn_steer_cost': 0.0064252, 'consistency losses/trn_te_loss': 23.33626513, 'aux losses/trn_pitch_loss': 1.18561935, 'aux losses/trn_yaw_loss': 0.23548985, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9609375, 'consistency losses/trn_torque_loss': 3045.18176079, 'consistency losses/trn_torque_delta_loss': 33.55173415, 'logistical/max_param': 16.22256279, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 194.2734375, 'logistical/obs_generated_per_second': 184.6640625, 'logistical/slowest_runner_obs_per_sec': 11.3671875, 'logistical/data_consumption_ratio': 1.05273107, 'logistical/manual_train_pause': 0.0, 'trn_c

KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e63.torch")

In [15]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
